# Graphing Code to Load Data and Plot It Live

## Imports

In [1]:
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
import datetime
import json
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import Processing_Functions_Tracking
from ast import literal_eval

## Initialising Dash and Plotly

In [2]:
app = app = dash.Dash(__name__)

#Plotly offline mode
init_notebook_mode(connected=True)

timer = 0

In [3]:
#Function to check if any of the items contain an array
def array_check(item,timer):
    item = item[0]
    print(item)
    if(isinstance(item, str)):
        print(item)
        item = literal_eval(item)
        return round(item[timer],2)
    elif(isinstance(item, list)):

        return round(item[-1],2)
    else:

        return round(float(item),2)

In [4]:
def processing_data(data):
    x_fitness = []
    y_fitness = []
    for item in data:
        print("Item: ",item)
        x_fitness.append(item[0])
        y_fitness.append(item[1])
    return x_fitness,y_fitness

In [5]:


path = "../Data for Report/Initial Deployment/Robot Array 100/100 Robots/"


store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
    dust = json.load(f)


constants = pd.read_excel(path + "Constants.xlsx", index_col=0)
min_neighbours = np.load(path + "Minimum Distance to Neighbours.npy")
cluster_average = np.load(path + "Cluster Average.npy")
total_collision = np.load(path + "Measurement Events Count.npy")

robot_number = constants.loc["Number of Robots"]
R = constants.loc["Communication Range"]
max_speed = constants.loc["Max Speed"]
G = constants.loc["G"]
power = constants.loc["Power"]
max_force = constants.loc["Max Force"]

#getting end positions
timer = 2000
robot_end = store_robots[:,:,timer-1]
x_positions = robot_end[0]
y_positions = robot_end[1]

length = 10
maximum = 2000
title = "Typical Deployment"
figure = Processing_Functions_Tracking.graph_grid_style([0],[0],500,10,title)
figure.show()

Maximum:  500
Length:  10
Maximum/Length:  100.0


path = "../Data for Report/Initial Deployment/"
average_minimum_neighbour = np.load(path + "Average Minimum Neighbour Distance for Different Deployments 2.npy")
print("Average Minimimum Neighbour: ",average_minimum_neighbour)
current_deployment =  average_minimum_neighbour[0]
deployment_range = (0+1)*10
deployment_legend = "Initialisation between " + str(-deployment_range)+ " and " + str(deployment_range)
print("Average Minimimum Neighbour " + str(0) + ": ",current_deployment)
x,y = processing_data(current_deployment)
print("Processed Data: ")
performance = Processing_Functions_Tracking.graph_figure_fitness_max_report(np.array(x),np.array(y),"Number of Robots","Average Minimum Neighbour Distance (m)","",10,200,deployment_legend)
performance.show()

for i in range(1,10):
    current_deployment =  average_minimum_neighbour[i]
    deployment_range = (i+1)*10
    name = "Initialisation between " + str(-deployment_range)+ " and " + str(deployment_range)
    print("Average Minimimum Neighbour " + str(i) + ": ",current_deployment)
    x_fitness,y_fitness = processing_data(current_deployment) 
    performance.add_trace(go.Scatter(
        x=x_fitness,
        y=y_fitness,
        mode = 'lines+markers',
        name = name,
        line=dict(width=4,
                              dash='dash'),
         marker=dict(
            symbol='circle',
            opacity=0.6,
            size=8,
            line=dict(
                color='black',
                width=1
            ))
        ))

    print("Processed Data: ")
#performance.add_trace(Processing_Functions_Tracking.graph_figure_fitness_max_report(np.array(x),np.array(y),"Number of Robots","Average Minimum Neighbour Distance (m)","",10,200,"Deployment"))
performance.show()

# Loading Data

In [6]:
#code = "G10392.0"
#path = "Experiments/100 Centre" + "/100 Robots/" + code + "/"
#path = path = "../Experiments/Detecting Dust Devils - Passive Survey 1. Final Deployment Sweep with Dust Devils/50 Robots/"
"""global count
global path"""
static = "Full Day Static 10-50/"
tracking = "Tracking_Day_Long/"
tracking_no_G = "Tracking_Day_No_G_Broadcast/" #Stres
base = "../Blue Pebble/Files/"
tracking_increased_decay = "Tracking_Day_Increased_Decay/"
healing_path = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And No Broadcasting G. Checking Healing of 100 Robots/100 Robots/"
path = base+tracking_increased_decay + "100 Robots/Run_0/"#"../Data for Report/Detecting Dust Devils - Testing Tracking Functionality with R Added Dust Probability 4.0/70 Robots/"
print(path)
store_robots = np.load(path + "Robots.npy")

with open(path + "dust.txt", "r") as f:
    dust = json.load(f)


constants = pd.read_excel(path + "Constants.xlsx", index_col=0)
min_neighbours = np.load(path + "Minimum Distance to Neighbours.npy")
cluster_average = np.load(path + "Cluster Average.npy")
total_collision = np.load(path + "Measurement Events Count.npy")
total_detection = np.load(path + "Number of Dust Devils Detected.npy")
total_dust = np.load(path + "Number of Dust Devils Generated.npy")
print(total_dust)
robot_number = constants.loc["Number of Robots"]
R = constants.loc["Communication Range"]
max_speed = constants.loc["Max Speed"]
G = constants.loc["G"]
power = constants.loc["Power"]
max_force = constants.loc["Max Force"]

title = ""

maximum = 500
tick_space = maximum/20
healing_title = "<b>Tracking Method 2 with Increased Decay Time and Broadcast G</b>"
title = "Tracking with Triangular Physicomimetics Algorithm"
timer = 0
count = 0

../Blue Pebble/Files/Tracking_Day_Increased_Decay/100 Robots/Run_0/
[ 0  0  0 ... 98 98 98]


In [7]:
"""path1 = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And No Broadcasting G. Checking Healing of 100 Robots/100 Robots/"
path2  = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And Broadcasting G. Checking Healing of 100 Robots/100 Robots/""
load(path)"""

'path1 = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And No Broadcasting G. Checking Healing of 100 Robots/100 Robots/"\npath2  = "../Experiments/Detecting Dust Devils - Testing Tracking Functionality by using Clustering Algorithm And Broadcasting G. Checking Healing of 100 Robots/100 Robots/""\nload(path)'

In [8]:

#the update layout function
def update_layout():
    #contains the graph, the interval timing and the live updating table
    app.layout = html.Div(children=[html.Div(
    [
    dcc.Graph(id = 'live_graph'),
    dcc.Interval(
        id = 'graph-update',
        interval = 100, #milliseconds
        n_intervals = 0
        ),
    ],style = {'display': 'inline-block'} ),
        html.Div(
            [
                dcc.Graph(id = 'live_table'),
                dcc.Interval(
                id = 'variable-update',
                interval = 100, #milliseconds
                n_intervals = 0
                )    
                 ],
        style = {'display': 'inline-block','vertical-align': 'top' }    
    )],style={'width': '100%', 'display': 'inline-block'})

update_layout()

In [9]:

#callback is the updating process
@app.callback(Output('live_graph','figure'),
              [Input('graph-update', 'n_intervals')])

#the actual update for the graph
def update_graph(n):
    #initialising global variables
    global timer
    global count 
    
    """if(timer>4999):
        timer = 0"""
    search = timer%5 
    position_robot = store_robots[:,:,timer]
    position_dust = dust[timer]
    x_dust = position_dust[0]
    y_dust = position_dust[1]
    #fig = Processing_Functions_Tracking.graph_grid_style(position_robot[0],position_robot[1],2000,10,title)
    #creating scatter plot of robots
    data = go.Scatter(
        x=list(position_robot[0]),
        y=list(position_robot[1]),
        name = 'Robots       ',
        mode = 'markers',
    )
    
    #creating the plotly figure with the robot data
    fig = go.Figure(
        { "data": data, "layout": go.Layout(yaxis=dict(range=[-maximum, maximum]),xaxis = dict(range=[-maximum,maximum]))
        }
    )
    
    fig.add_trace(go.Scatter(
          x= position_dust[0],
          y= position_dust[1],
          name='Dust Devil',
          mode = 'markers',
          marker_color='rgba(221,85,39, .9)',
      ))
    
    #updating layout with circles and different formatting'''
    fig.update_layout(title=healing_title,
    title_x=0.5,
    xaxis_title="X Position (m)",
    yaxis_title="Y Position (m)",
    margin=dict(
        t=50, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
         # bottom margin: 10px
        l=10, # left margin: 10px
        r=10, # right margin: 10px
    ),
    height=900,width=1150,
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = tick_space
    ),
                      yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = tick_space
    ),  showlegend = True
                     )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black',mirror=True)
    #incrementing timer
    if(timer == 0 and count < 50):
        timer = 0
        count = count+1
    else:
        timer = timer + 5
    

    #updating the layout
    update_layout()
    return fig

#app callback for the table
@app.callback(Output('live_table','figure'),
              [Input('variable-update', 'n_intervals')])

def update_variables(n):
    fig=Processing_Functions_Tracking.table_figure_dash(timer,robot_number,R,G,power,max_force,max_speed,min_neighbours[timer],cluster_average[timer])
         #fig.layout['template']['data']['table'][0]['header']['fill']['color']='#636EFA'
    fig.update_layout(width = 650, height = 1500,margin=dict(
    t=260, # top margin: 30px, you want to leave around 30 pixels to
              # display the modebar above the graph.
        b=10, # bottom margin: 10px
        l=10, # left margin: 10px
        r=10,
    ))
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
    return fig

In [ ]:
#runs by default
if __name__ == '__main__':
    #running the app server for the simulator
    app.run_server(debug=False, host = '127.0.0.1')
    #app.run_server(debug=False,port=8080,host='0.0.0.0')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 

127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:31:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.

...

127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:29] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:40:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [12/Sep/2021 22:41:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 22:41:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.

## 